In [47]:
import kiutils.symbol
import random
import os
import glob
import pandas as pd

#### Function for generating a unique BR ID ####
def generate_BRID(existing_BRIDs):

    # Set number of digits in unique BRID
    num_digits = 6      

    # Increment by 1 until the BRID doesn't exist
    for n in range(10**num_digits):

        # Convert to a zero padded string (to fill the desired number of digits -- e.g, 1 --> '0001')
        BR_num = str(n).zfill(num_digits)
        BRID = 'BRE-' + BR_num

        # If the new BRID isn't currently used, we've found the one!
        if BRID not in existing_BRIDs:
            break

    return BRID

def add_field_to_symbol(symbol_lib, symbol_name, field_name, field_value):
    """
    Add a custom field to a KiCad symbol (.kicad_sym file) using kiutils.

    Args:
        symbol_lib (kiutils SymbolLib): symbol_lib object being read
        symbol_name (str): The name of the symbol to modify.
        field_name (str): The name of the field to add.
        field_value (str): The value of the field to add.
    """

    # Find the symbol by name
    symbol = None
    for sym in symbol_lib.symbols:
        if sym.libId == symbol_name:
            symbol = sym
            break

    if symbol is None:
        raise ValueError(f'Symbol "{symbol_name}" not found in the library.')

    # Create the new field using kiutils' built-in property structure
    field_id = len(symbol.properties)  # Set ID to the next available index
    new_field = kiutils.symbol.Property(key=field_name, value=field_value, id=field_id)

    # Add the new field to the symbol's properties
    symbol.properties.append(new_field)

    # Save the modified symbol back to the file
    print(f'Field "{field_name}" with value "{field_value}" added to symbol "{symbol_name}".')


In [48]:
parts_df = pd.DataFrame(columns=['ID','Name','Description','Value','Symbol','Footprint','Datasheet','Manufacturer','MPN', 'Category'])
vendors_df = pd.DataFrame(columns=['ID','Supplier','SPN','Stock'])

ID_list = []
parts_list = []
vendors_list = []

# Reading symbol libraries from our BR symbols folder
SYMBOLS_PATH = "C:/Users/JacobBrotmanKrass/Documents/GitHub/br-kicad-lib/Symbols"
os.chdir(SYMBOLS_PATH)
for lib_file in glob.glob("*.kicad_sym"):

    # Extract library nickname/category -- e.g., 0402_Capacitors
    lib_nickname = lib_file.replace(".kicad_sym", "")

    # Skip these libraries, they don't need to be documented (obsolete or not actual parts)
    if (lib_nickname == "BR~Deprecated") or (lib_nickname == "BR_Virtual_Parts"):
        continue
    print(lib_nickname)

    # Open symbol library
    lib_path = os.path.join(SYMBOLS_PATH, lib_file)
    symbol_lib = kiutils.symbol.SymbolLib().from_file(lib_path)
    
    # The Category is the library nickname, without the BR_ at the beginning
    category = lib_nickname[3:]

    # For each symbol in a given library, populate a new row in the Parts dataframe
    for symbol in symbol_lib.symbols:

        # Symbol path in Kicad
        symbol_path = f"{lib_nickname}:{symbol.entryName}"

        # Generate a unique BR ID --- Only necessary for this first commit of Kicad parts into our BR "database"
        BR_ID = generate_BRID(ID_list)
        ID_list.append(BR_ID)

        # Grab all the properties from the Kicad Symbol
        properties = {property.key: property.value for property in symbol.properties}

        # Remove a weird decoding error with the plus/minus sign
        bad_char = b'\xc3\x82'.decode()
        properties["Description"] = properties["Description"].replace(bad_char, '')

        # Some parts don't have a manufacturer and manufacturer part number -- deal with this some other time, for now just populate "None"
        if "Manufacturer" in properties:
            manufacturer = properties["Manufacturer"]
            mpn = properties["Manufacturer Part Num"]
        else:
            manufacturer = "None"
            mpn = "None"

        # Append a dictionary of all part properties to the parts list -- this will be converted to a Pandas dataframe at the end
        parts_list.append({"BR ID":BR_ID, "Name":symbol.libId, "Description":properties["Description"], "Value":properties["Value"], "Symbol":symbol_path, "Footprint":properties["Footprint"],  "Datasheet":properties["Datasheet"], "Manufacturer":manufacturer, "MPN":mpn, "Category":category})

        # Add BR ID to the symbol
        add_field_to_symbol(symbol_lib, symbol.libId, "BR ID", BR_ID)

        # Extract all supplier-related properties: supplier X with suppler number X
        supplier_properties = {property: properties[property] for property in properties if property[:8]=="Supplier"}
        supplier_numbers = {supp_prop: supplier_properties[supp_prop] for supp_prop in supplier_properties if supp_prop[9]=='P'}
        supplier_names = {supp_prop: supplier_properties[supp_prop] for supp_prop in supplier_properties if supp_prop[9]!='P'}

        # Ignore any thing that looks like this
        null_strings = ["", " ", "-", "--", "~", "NA", "N/A"]

        # Loop through and add vendors and the respective supplier number when the number X at the end matches (supplier 1 --> supplier part num 1)
        for name in supplier_names:
            for number in supplier_numbers:
                if name[-1] == number[-1]:
                    if supplier_numbers[number] not in null_strings:

                        # Append dictionary of supplier properties for each SPN to the vendors list
                        # this will be converted to Pandas dataframe at the end
                        vendors_list.append({"BR ID":BR_ID, "Supplier":supplier_names[name], "SPN":supplier_numbers[number], "Stock":0})

    symbol_lib.to_file()

# Create Pandas dataframes from these lists of dictionaries
# Think of each dictionary as a row in the table
parts_df = pd.DataFrame(parts_list)
vendors_df = pd.DataFrame(vendors_list)

# Create dataframe from the JLC scrape spreadsheet
jlc_df = pd.read_excel(r"C:\Users\JacobBrotmanKrass\Documents\GitHub\br-components-database\jlc-scraper\csv\Parts Inventory on JLCPCB.xlsx")

# Merge the vendors and JLC dataframes by Supplier Part Number
jlc_df.rename(columns={"JLCPCB Part #":"SPN"}, inplace=True)
merged_df = pd.merge(vendors_df.set_index("SPN"), jlc_df.set_index("SPN"), on="SPN", how='left')

# Populate the stock column using the maximum value of the three sources of JLC stock
merged_df["Stock"] = merged_df[["JLCPCB Parts Qty", "Global Sourcing Parts Qty", "Consigned Parts Qty"]].max(axis=1)

# I think we only want the BR ID, Supplier, SPN, and Stock columns
vendor_stock_df = merged_df.reset_index()[["BR ID", "Supplier", "SPN", "Stock"]]

# Create a hierarchical index using the BR ID and its various associated supplier part numbers 
vendor_stock_df.set_index(["BR ID", "SPN"], inplace=True)

# Save dataframes to excel files
vendor_stock_df.to_excel("C:/Users/JacobBrotmanKrass/Documents/Vendor_Stock.xlsx")
parts_df.to_excel("C:/Users/JacobBrotmanKrass/Documents/Parts_Library.xlsx")

BR_Capacitors_0201
Field "BR ID" with value "BRE-000000" added to symbol "C_0201_100nF_25V_X7R_10%".
Field "BR ID" with value "BRE-000001" added to symbol "C_0201_10nF_25V_X7R_10%".
Field "BR ID" with value "BRE-000002" added to symbol "C_0201_1nF_50V_C0G_1%".
Field "BR ID" with value "BRE-000003" added to symbol "C_0201_1uF_16V_X5R_10%".
Field "BR ID" with value "BRE-000004" added to symbol "C_0201_220nF_25V_X5R_10%".
Field "BR ID" with value "BRE-000005" added to symbol "C_0201_3p9F_50V_C0G_5%".
Field "BR ID" with value "BRE-000006" added to symbol "C_0201_470nF_6V3_X5R_10%".
Field "BR ID" with value "BRE-000007" added to symbol "C_0201_470pF_50V_C0G_5%".
Field "BR ID" with value "BRE-000008" added to symbol "C_0201_4n7F_25V_X7R_10%".
Field "BR ID" with value "BRE-000009" added to symbol "C_0201_6pF_50V_C0G_15%".
BR_Capacitors_0402
Field "BR ID" with value "BRE-000010" added to symbol "C_0402_100nF_50V_X7R_10%_AEC-Q200_mini".
Field "BR ID" with value "BRE-000011" added to symbol "C_0

C:\Users\JacobBrotmanKrass\.conda\envs\condaenv\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [3]:
jlc_df = pd.read_excel(r"C:\Users\JacobBrotmanKrass\Documents\GitHub\br-components-database\jlc-scraper\csv\Parts Inventory on JLCPCB.xlsx")

C:\Users\JacobBrotmanKrass\.conda\envs\condaenv\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [51]:
parts_df.set_index("BR ID").to_excel("C:/Users/JacobBrotmanKrass/Documents/Parts_Library.xlsx")

In [46]:
parts_df

,ID,Name,Description,Value,Symbol,Footprint,Datasheet,Manufacturer,MPN,Category
0,BRE-000000,C_0201_100nF_25V_X7R_10%,100nF ±10% 25V X7R 0201 Ceramic Capacitor,100nF 25V,BR_Capacitors_0201:C_0201_100nF_25V_X7R_10%,BR_Passives:C_0201_0603Metric-minimized,https://www.lcsc.com/datasheet/lcsc_datasheet_...,Murata Electronics,GRM033R61E104KE14J,Capacitors_0201
1,BRE-000001,C_0201_10nF_25V_X7R_10%,10nF ±10% 25V X7R 0201 Ceramic Capacitor,10nF 25V,BR_Capacitors_0201:C_0201_10nF_25V_X7R_10%,BR_Passives:C_0201_0603Metric-minimized,https://search.murata.co.jp/Ceramy/image/img/A...,Murata Electronics,GRM033R71E103KE14D,Capacitors_0201
2,BRE-000002,C_0201_1nF_50V_C0G_1%,1nF ±1% 50V C0G 0201 Ceramic Capacitor,1nF 50V 1%,BR_Capacitors_0201:C_0201_1nF_50V_C0G_1%,BR_Passives:C_0201_0603Metric-minimized,https://www.lcsc.com/datasheet/lcsc_datasheet_...,Murata Electronics,GRM0335C1H102JE01D,Capacitors_0201
3,BRE-000003,C_0201_1uF_16V_X5R_10%,1uF ±10% 16V X5R 0201 Ceramic Capacitor,1uF 16V,BR_Capacitors_0201:C_0201_1uF_16V_X5R_10%,BR_Passives:C_0201_0603Metric-minimized,https://www.mouser.com/datasheet/2/281/1/GRM03...,Murata Electronics,GRM033R61C105ME15D,Capacitors_0201
4,BRE-000004,C_0201_220nF_25V_X5R_10%,220nF ±10% 25V X5R 0201 Ceramic Capacitor,220nF 25V,BR_Capacitors_0201:C_0201_220nF_25V_X5R_10%,BR_Passives:C_0201_0603Metric-minimized,https://www.lcsc.com/datasheet/lcsc_datasheet_...,Murata Electronics,GRM033R61E224KE01D,Capacitors_0201
...,...,...,...,...,...,...,...,...,...,...
471,BRE-000471,TP_SQUARE_SMT_3.4mm_x_1.8mm,"PC Test Point, Miniature Phosphor Bronze Silve...",5017,BR_Virtual_Parts:TP_SQUARE_SMT_3.4mm_x_1.8mm,BR_Virtual_Parts:KEYSTONE_5017,https://www.keyelco.com/userAssets/file/M65p55...,Keystone Electronics,5017,Virtual_Parts
472,BRE-000472,TP_SQUARE_TH_1mm,test pad through hole,TP_SQUARE_TH_500um,BR_Virtual_Parts:TP_SQUARE_TH_1mm,BR_Virtual_Parts:TP_SQUARE_TH_1mm,~,~,~,Virtual_Parts
473,BRE-000473,TP_SQUARE_TH_2mm,"test pad, through hole",TP_SQUARE_TH_2mm,BR_Virtual_Parts:TP_SQUARE_TH_2mm,BR_Virtual_Parts:TP_SQUARE_TH_2mm,~,~,~,Virtual_Parts
474,BRE-000474,TP_SQUARE_TH_500um,test pad through hole,TP_SQUARE_TH_500um,BR_Virtual_Parts:TP_SQUARE_TH_500um,BR_Virtual_Parts:TP_SQUARE_TH_500um,~,~,~,Virtual_Parts


In [31]:
jlc_df = jlc_df.rename(columns={"JLCPCB Part #":"SPN"})
merged_df = pd.merge(vendors_df.set_index("SPN"), jlc_df.set_index("SPN"), on="SPN", how='left')

In [32]:

merged_df["Stock"] = merged_df[["JLCPCB Parts Qty", "Global Sourcing Parts Qty", "Consigned Parts Qty"]].max(axis=1)

In [33]:
merged_df.reset_index()

,SPN,ID,Supplier,Stock,Category,MFR Part #,Footprint,JLCPCB Parts Qty,Global Sourcing Parts Qty,Consigned Parts Qty
0,490-14571-1-ND,BRE-000000,DigiKey,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,81-GRM033R61E104KE4J,BRE-000000,Mouser,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,C2649540,BRE-000000,JLCPCB,5000.0,Capacitors,GRM033R61E104KE14J,0201,5000.0,0.0,0.0
3,490-14454-2-ND,BRE-000001,DigiKey,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,81-GRM0335C1E12FA1D,BRE-000001,Mouser,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
1000,C9900082548,BRE-000445,JLCPCB,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1001,C2916352,BRE-000446,JLCPCB,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1002,C2916351,BRE-000447,JLCPCB,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1003,36-5017CT-ND,BRE-000471,DigiKey,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [34]:
vendor_stock_df = merged_df.reset_index()[["ID", "Supplier", "SPN", "Stock"]]
vendor_stock_df.set_index(["ID", "SPN"], inplace=True)
vendor_stock_df

Supplier   Stock
ID         SPN                                  
BRE-000000 490-14571-1-ND        DigiKey     NaN
           81-GRM033R61E104KE4J   Mouser     NaN
           C2649540               JLCPCB  5000.0
BRE-000001 490-14454-2-ND        DigiKey     NaN
           81-GRM0335C1E12FA1D    Mouser     NaN
...                                  ...     ...
BRE-000445 C9900082548            JLCPCB     NaN
BRE-000446 C2916352               JLCPCB     NaN
BRE-000447 C2916351               JLCPCB     NaN
BRE-000471 36-5017CT-ND          DigiKey     NaN
           C238130                JLCPCB     NaN

[1005 rows x 2 columns]

In [27]:
vendor_stock_df

,ID,Supplier,SPN,Stock
0,BRE-000000,JLCPCB,C2649540,5000
1,BRE-000003,JLCPCB,C335102,1000
2,BRE-000005,JLCPCB,C76926,0
3,BRE-000007,JLCPCB,C3886751,1000
4,BRE-000009,JLCPCB,C161434,1000
...,...,...,...,...
122,BRE-000410,JLCPCB,C2921584,60
123,BRE-000413,JLCPCB,C720477,100
124,BRE-000419,JLCPCB,C154911,100
125,BRE-000426,JLCPCB,C102608,0


In [18]:
id = ['00', '01', '01', '02']
stock = [15, 12, 7, 0]
spn = ['C12','C19','C13','C29']
test_df = pd.DataFrame(data={"ID":id, "Stock":stock, "SPN":spn}).set_index(["ID", "SPN"])

In [38]:
vendor_stock_df

Supplier   Stock
ID         SPN                                  
BRE-000000 490-14571-1-ND        DigiKey     NaN
           81-GRM033R61E104KE4J   Mouser     NaN
           C2649540               JLCPCB  5000.0
BRE-000001 490-14454-2-ND        DigiKey     NaN
           81-GRM0335C1E12FA1D    Mouser     NaN
...                                  ...     ...
BRE-000445 C9900082548            JLCPCB     NaN
BRE-000446 C2916352               JLCPCB     NaN
BRE-000447 C2916351               JLCPCB     NaN
BRE-000471 36-5017CT-ND          DigiKey     NaN
           C238130                JLCPCB     NaN

[1005 rows x 2 columns]

In [44]:
vendor_stock_df.to_excel("C:/Users/JacobBrotmanKrass/Documents/output.xlsx")

In [45]:
read_df = pd.read_excel("C:/Users/JacobBrotmanKrass/Documents/output.xlsx", index_col=[0, 1])
read_df[read_df.Supplier == "JLCPCB"]

,,Supplier,Stock
ID,SPN,,
BRE-000000,C2649540,JLCPCB,5000.0
BRE-000001,C85930,JLCPCB,NaN
BRE-000002,C3870067,JLCPCB,NaN
BRE-000003,C335102,JLCPCB,1000.0
BRE-000004,C701263,JLCPCB,NaN
...,...,...,...
BRE-000444,C2916384,JLCPCB,NaN
BRE-000445,C9900082548,JLCPCB,NaN
BRE-000446,C2916352,JLCPCB,NaN
